In [8]:
import pandas as pd
from google.oauth2 import service_account
from google.cloud import bigquery
import pyodbc
import logging

In [47]:
from sqlalchemy import create_engine
import urllib
from sqlalchemy.types import Integer, String, Unicode, Boolean, DateTime, Float, Numeric

In [9]:
def save_bigquery(df, dataset, table):
    """Import a csv file into BigQuery"""
    logging.info(table)
    client = bigquery.Client()
    
    table_ref = client.dataset(dataset).table(table)
    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
    job_config.autodetect = True
    load_job = client.load_table_from_dataframe(df,
                                                table_ref,
                                                job_config=job_config)
    # Waits for table load to complete.
    load_job.result()

In [10]:
def connect(request):
    server = '10.34.1.4' 
    database = 'ODS_UNILEVER' 
    username = 'BIGQUERY_BI' 
    password = 'AN2 6optimist hurts a backpack' 
    cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password, timeout=login_timeout)
    cursor = cnxn.cursor()
    tableorigem = 'Pedido'
    sql = 'SELECT * FROM ODS_UNILEVER.infrashop.' + tableorigem
    table = tableorigem
    dataset = "infrashop_raw"
    df = pd.read_sql(sql,cnxn)
    
    logging.info(f'leu tabela = {table}')
    save_bigquery(df, dataset, table)
    logging.info(f'gravou tabela = {table}')


In [13]:
server = '10.34.1.4' 
database = 'ODS_UNILEVER' 
username = 'BIGQUERY_BI' 
password = 'AN2 6optimist hurts a backpack' 
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
tableorigem = 'Pedido'
sql = 'SELECT * FROM ODS_UNILEVER.infrashop.' + tableorigem
table = tableorigem
dataset = "infrashop_raw"
df = pd.read_sql(sql,cnxn)

In [16]:
df.head()

PedidoId  MasterPedidoId  ClienteId  EnderecoEntregaId  CupomDescontoId  \
0         1               1       4370               4370                0   
1  10519134        10519137      11213              11213                0   
2  10521120        10521123      17990              17990                0   
3  10547235        10547238      20644              20644                0   
4  10548078        10548081      10907              10907                0   

   ValorTotal  ValorTotalFrete  ValorCupomDesconto  ValorCupomDescontoFrete  \
0      333.60              0.0                 0.0                      0.0   
1     3853.36              0.0                 NaN                      NaN   
2      513.82              0.0                 NaN                      NaN   
3      439.07              0.0                 NaN                      NaN   
4      723.88              0.0                 NaN                      NaN   

                IP  ... SistemaOperacional PrazoCd  Marketplace  Fingerprint  \
0  170.238.117.25,  ...            Desktop       0            1   3711524516   
1   186.225.84.192  ...               None       0            0         None   
2    177.37.142.92  ...               None       0            0         None   
3     201.20.109.2  ...               None       0            0         None   
4    170.83.67.251  ...               None       0            0         None   

   SegmentoClienteId                                      HASH  \
0               18.0  03ac23b369cd48a89d2ab3dca60161db3060960a   
1                NaN  fa60fe0848b69c1aa3ad6c66e0704b59ca876d33   
2                NaN  547c8f563363e83cf214974428bed28d7b5f6dd1   
3                NaN  e1e6bcc3c18186c147aab424c9793ccce81cc342   
4                NaN  92477d9b751f33d3236b34e959f42b83e9cdf11c   

                DAT_CARGA  PrazoEntregaTexto  CodigoPoliticaPreco  Migracao  
0 2020-02-29 19:44:20.327               None                 None       NaN  
1 2020-03-04 05:54:47.560               None                 None       1.0  
2 2020-03-04 05:54:47.560               None                 None       1.0  
3 2020-03-04 05:54:47.560               None                 None       1.0  
4 2020-03-04 05:54:47.560               None                 None       1.0  

[5 rows x 33 columns]

In [43]:
server = '10.34.1.4' 
database = 'AMB_BIGQUERY' 
username = 'BIGQUERY_BI' 
password = 'AN2 6optimist hurts a backpack' 
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [28]:
for index,row in df.iterrows():
    #sql = "INSERT INTO Teste_BigQuery([PedidoId],[MasterPedidoId],[ClienteId]) values (?, ?, ?)"
    cursor.execute("INSERT INTO infrashop.Teste_BigQuery([PedidoId],[MasterPedidoId],[ClienteId]) values (?, ?, ?)", 1, 2, 3)

ProgrammingError: ('42S02', "[42S02] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Invalid object name 'infrashop.Teste_BigQuery'. (208) (SQLExecDirectW)")

In [37]:
cursor.execute("CREATE TABLE BI_UNILEVER.dw.NewTable ( Column1 varchar(100) NULL ) GO")


ProgrammingError: ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Incorrect syntax near 'GO'. (102) (SQLExecDirectW)")

In [32]:
df.to_sql(name = 'Teste_BigQuery2', con = cnxn, if_exists = 'replace', index = False)

DatabaseError: Execution failed on sql 'SELECT name FROM sqlite_master WHERE type='table' AND name=?;': ('42S02', "[42S02] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Invalid object name 'sqlite_master'. (208) (SQLExecDirectW)")

In [45]:
db_params = urllib.parse.quote_plus('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
engine = create_engine("mssql+pyodbc:///?odbc_connect={}".format(db_params))

In [ ]:
df.to_sql('Teste1',con=engine,index=False,if_exists="replace",schema="dbo")

In [50]:
def get_data(dataproject, datasetprojectbi, table, clausule = ""):
    query = f"SELECT * FROM `{dataproject}.{datasetprojectbi}.{table}` {clausule}"
    datatable = pd.read_gbq(
            query,
            project_id="bigqueryunilever",
            dialect="standard")
    return datatable

In [51]:
df1 = get_data("bigqueryunilever", "PED_SKU", "Calculo_6_Qtd_Item_A_Item", clausule = "")

Downloading: 100%|██████████| 1544340/1544340 [12:17<00:00, 2094.17rows/s]


In [52]:
df1.head()

PedidoId  ProdutoId  EstabelecimentoId  ReservaId  PedidoProdutoGarantiaId  \
0  60001927      10037                NaN        NaN                      NaN   
1  60002285       6130                NaN        NaN                      NaN   
2  60005782      10306                NaN        NaN                      NaN   
3  60000573      10087                NaN        NaN                      NaN   
4  60004949       4688                NaN        NaN                      NaN   

   PedidoProdutoSeguroId  ListaId ProdutoCodigo  Quantidade  ValorUnitario  \
0                    NaN      NaN          None         NaN            NaN   
1                    NaN      NaN          None         NaN            NaN   
2                    NaN      NaN          None         NaN            NaN   
3                    NaN      NaN          None         NaN            NaN   
4                    NaN      NaN          None         NaN            NaN   

   ...  HASH                        DAT_CARGA  Linhas_PED_SKU_KIT  \
0  ...  None 2020-03-08 19:25:32.077000+00:00                 NaN   
1  ...  None 2020-03-08 19:25:32.077000+00:00                 NaN   
2  ...  None 2020-03-08 19:25:32.077000+00:00                 NaN   
3  ...  None 2020-03-08 19:25:32.077000+00:00                 NaN   
4  ...  None 2020-03-08 19:25:32.077000+00:00                 NaN   

   KeyPedidoProduto  SumBonificadoPedidoSKU Kit_Bonificado  \
0              None                     NaN           None   
1              None                     NaN           None   
2              None                     NaN           None   
3              None                     NaN           None   
4              None                     NaN           None   

   KeyPedidoMetaSKUId  KeyPedidoProdutoBonificado  QTD_FATURADO_ITEM_A_ITEM  \
0                None                        None                       NaN   
1                None                        None                       NaN   
2                None                        None                       NaN   
3                None                        None                       NaN   
4                None                        None                       NaN   

   Min_Soma  
0       NaN  
1       NaN  
2       NaN  
3       NaN  
4       NaN  

[5 rows x 52 columns]